In [1]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import tempfile

In [2]:
# Function to get keypoint correspondences between frames and the book image
def get_correspondences(frame1, book_img, num_correspondences=50):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(book_img, None)
    kp2, des2 = sift.detectAndCompute(frame1, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.25 * n.distance:
            good_matches.append(m)

    good_matches = good_matches[:num_correspondences]

    # Save the correspondences 
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
   
    return pts1, pts2

In [3]:
# Function to compute homography matrix from point correspondences
def compute_homography(pts1, pts2):
    A = []
    for i in range(len(pts1)):
        x, y = pts1[i][0], pts1[i][1]
        u, v = pts2[i][0], pts2[i][1]
        A.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])

    A = np.asarray(A)
    U, S, Vh = np.linalg.svd(A)
    H = Vh[-1, :].reshape(3, 3)
    # Normalize H (optional)
    H /= H[2, 2]
    return H

In [4]:
# Function to map points using a homography matrix
def map_points_with_homography(H, points):
    points_homogeneous = np.hstack((points, np.ones((len(points), 1))))
    mapped_points_homogeneous = np.dot(H, points_homogeneous.T)
    # Normalize homogeneous coordinates
    mapped_points = mapped_points_homogeneous[:2, :] / mapped_points_homogeneous[2, :].reshape(1, -1)
    return mapped_points.T

In [5]:
# RANSAC to find the best homography using inliers
def RANSAC(pts1, pts2, num_iterations=1000, min_set_size=4, inlier_threshold=.5, min_inliers=45):
    best_H = None
    max_inliers = 0
    num_correspondences = len(pts1)
    np.random.seed(42)
    for i in range(num_iterations):
        random_indices = np.random.choice(num_correspondences, size=min_set_size, replace=False)
        sampled_pts1 = pts1[random_indices]
        sampled_pts2 = pts2[random_indices]

        initial_H = compute_homography(sampled_pts1, sampled_pts2)
        transformed_points = map_points_with_homography(initial_H, pts1)

        errors = np.sqrt(np.sum((transformed_points - pts2)**2, axis=1))
        inliers = np.sum(errors < inlier_threshold)

        if inliers > max_inliers:
            max_inliers = inliers
            best_H = initial_H

        if max_inliers > min_inliers:
            break

    return best_H

In [6]:
# Function to crop and fit the AR video frame to the book region
def crop_ar_video_frame(video_frame, book_corners):
    
    # Calculate the target width and height from book corners
    book_width = int(book_corners[1][0] - book_corners[0][0])
    book_height = int(book_corners[3][1] - book_corners[0][1])

    # Resize the AR video frame to exactly match the book region
    resized_frame = cv2.resize(video_frame, (book_width, book_height), interpolation=cv2.INTER_AREA)

    return resized_frame


In [7]:
# Function to overlay the frames of the book and AR video
def overlay_frames(frame1, frame2, H, book_corners):
    book_coordinates_video = map_points_with_homography(H, book_corners)

    mask = np.zeros_like(frame1, dtype=np.uint8)
    cv2.fillPoly(mask, [np.int32(book_coordinates_video)], (255, 255, 255))

    inverted_mask = cv2.bitwise_not(mask)

    frame1_blacked = cv2.bitwise_and(frame1, inverted_mask)

    overlay_frame = cv2.warpPerspective(frame2, H, (frame1.shape[1], frame1.shape[0]))

    result = cv2.add(frame1_blacked, overlay_frame)

    return result

In [8]:
# Function to process video and overlay AR onto book
def process_video(video1_path, video2_path, book_img_path):
    
    # Open the video files
    video1 = cv2.VideoCapture(video1_path)
    video2 = cv2.VideoCapture(video2_path)

    # Load the book image
    book_img = cv2.imread(book_img_path)

    if book_img is None:
        raise ValueError("Image not found at path: " + book_img_path)

    # Set up the video writer
    width = int(video1.get(3))
    height = int(video1.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    
    # Define the output path in the same directory
    output_path = 'output_video_edited.avi'
    
    output_video = cv2.VideoWriter(output_path, fourcc, 30.0, (width, height))

    # Define the book corners
    book_corners = np.array([[0, 0],
                             [book_img.shape[1] - 1, 0],
                             [book_img.shape[1] - 1, (book_img.shape[0] - 1)],
                             [0, (book_img.shape[0] - 1)]],
                            dtype=np.float32)

    # Main processing loop
    while True:
        ret1, frame1 = video1.read()
        ret2, frame2 = video2.read()

        if not ret1 or not ret2:
            break

        # Get correspondences for each frame
        pts_book, pts_video = get_correspondences(frame1, book_img)

        # Calculate homography matrix
        H = RANSAC(np.squeeze(pts_book), np.squeeze(pts_video))

        # Crop the video frame centered on the book
        cropped_video_frame = crop_ar_video_frame(frame2, book_corners)

        # Overlay frames and write to the output video
        result_frame = overlay_frames(frame1, cropped_video_frame, H, book_corners)
        output_video.write(result_frame)

    # Release video captures and writer
    video1.release()
    video2.release()
    output_video.release()

    return output_path

In [9]:
video1_path = "book.mov"
video2_path = "ar_source_edited.mov"
book_img_path = "cv_cover.jpg"

output_path = process_video(video1_path, video2_path, book_img_path)

print(f"Output video saved to: {output_path}")

Output video saved to: output_video_edited.avi
